## Multiple Items

**Goals** beyond approach for single items:

* manage _coordination_ of ordering
* manage _common constraints_ (budget, space, etc.)
* allow _consolidation_ (shared costs, etc.)
* _conserve management time_

**Techniques**:

* Aggregate SKUs under similar operating policies
    * group via common characteristics or break points
    * group via Power of Two policies
* Manage inventory under common constraints - Exchange Curves
    * for Cycle Stock
    * for Safety Stock

### Group Like Items - Break Points

**Basic Idea**:

Replenish higher value items faster

**Usage**: 

Multiple items with:

* (relatively) stable demand
* *similar* ordering costs, $c_t$
* *similar* holding charges, $h$
* *different* annual demands, $D_i$
* *different* purchase costs, $c_i$

**Approach**:

* let $i$ represent the items under consideration
* pick base time period, $w_0$
    * typically a week
* define set of candidate ordering periods, $w_j$: ($w_1$, $w_2$, etc.)
    * note that the periods must be increasing, s.t. $w_{j-1} < w_j < w_{j+1}$
* find $D_ic_i$ values where $TRC(w_j) = TRC(w_{j+1})$
* group SKUS that fall in common value ($D_ic_i$) buckets

**Rule**:

```
if   D_i * c_i >= 5408 * c_t / (h * w_1 * w_2) then select w_1
elif D_i * c_i >= 5408 * c_t / (h * w_2 * w_3) then select w_2
elif D_i * c_i >= 5408 * c_t / (h * w_3 * w_4) then select w_3
etc ...
```

Note: The value '5408' assumes all costs and demand are defined in annual terms, and the $w_j$ values are _weekly_ increments.  It's calculated as $2 \cdot 52^2$

In [3]:
# TODO: concrete example
# TODO: plot showing how different SKUs fall into groups

### Power of Two Policies

**Motivation**: 

* (from EOQ lessons):
* TRC will be within 6% of optimal when ordering in time intervals of powers of two

**Technique**:

* create table of SKUs
* calculate $T^*$ for each SKU

$$
T^* = \frac{Q^*}{D} \
    = \frac{\sqrt{ \frac{2 c_t D}{c_e} }}{D} \
    = \sqrt{ \frac{ 2 c_t } { D c_e }  }
$$

* calculate $T_{practical}$ for each SKU

$$
p = \Biggl\lceil \frac{\ln{\left( \frac{T^*}{\sqrt{2}} \right) }}{\ln{(2)}} \Biggr\rceil
$$

$$
T_{practical} = 2^{p}
$$

In [4]:
# TODO: concrete example
# TODO: plot showing how different SKUs fall into different groups

### Exchange Curves - Cycle Stock

**Motivation**:

* under a given budget for inventory
* find the best allocation across multiple SKUs
* using cost parameters as policy 'levers'

**Relevant Cost Parameters**:

* holding charge, $h$
* ordering cost, $c_t$

**Exchange Curve**:

* trade-off between TACS and N:
    * total annual cycle stock (TACS)
    * $\sum_{i=1}^n \frac{Q_ic_i}{2}$    
    * number of replenishments (N)
    * $\sum_{i=1}^n \frac{D_i}{Q_i}$
* determine $\frac{c_t}{h}$ that meets budget constraints